In [2]:
import tensorflow as tf
import keras
import numpy as np
import cv2
import glob
from PIL import Image , ImageDraw
import os
import xmltodict

In [3]:
input_dim = 228

In [4]:
images = []
image_paths = glob.glob("training_images/*.jpg")

In [5]:
for i in image_paths:
    image = Image.open(i).resize((input_dim, input_dim))
    # Normalize image
    image = np.asarray(image) / 255.0
    images.append(image)

In [19]:
x

OrderedDict([('annotation',
              OrderedDict([('folder', 'single mushroom'),
                           ('filename', 'mushroom_30.jpg'),
                           ('path',
                            'C:\\Users\\Muhammed Buyukkinaci\\Downloads\\single mushroom\\mushroom_30.jpg'),
                           ('source', OrderedDict([('database', 'Unknown')])),
                           ('size',
                            OrderedDict([('width', '227'),
                                         ('height', '227'),
                                         ('depth', '3')])),
                           ('segmented', '0'),
                           ('object',
                            OrderedDict([('name', 'mushroom'),
                                         ('pose', 'Unspecified'),
                                         ('truncated', '0'),
                                         ('difficult', '0'),
                                         ('bndbox',
                           

In [28]:
x['annotation']['object']['bndbox']['xmin']

'9'

In [36]:
xml_paths = glob.glob("training_images/*.xml")
objects = []
bndboxes = []
for path in xml_paths:
    annotation_dict = xmltodict.parse(open(path, 'rb'))
    objects.append(annotation_dict['annotation']['object']['name'])
    bb = []
    buff = annotation_dict['annotation']['object']['bndbox']
    bb.append(int(buff['xmin']))
    bb.append(int(buff['ymin']))
    bb.append(int(buff['xmax']))
    bb.append(int(buff['ymax']))
    bb = np.asarray(bb) / input_dim
    bndboxes.append(bb)

In [38]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [47]:
boxes = np.array(bndboxes)
encoder = LabelBinarizer()
classes_one_hot = encoder.fit_transform(objects)
Y = np.concatenate([boxes, classes_one_hot], axis = 1)
X = np.asarray(images)

In [60]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, train_size = 0.6)

In [62]:
# Creating the keras model

In [65]:
from tensorflow import keras
from tensorflow.keras import backend as K

In [66]:
input_shape = ( input_dim , input_dim , 3 )
dropout_rate = 0.5
alpha = 0.2

def calculate_iou( target_boxes , pred_boxes ):
    xA = K.maximum( target_boxes[ ... , 0], pred_boxes[ ... , 0] )
    yA = K.maximum( target_boxes[ ... , 1], pred_boxes[ ... , 1] )
    xB = K.minimum( target_boxes[ ... , 2], pred_boxes[ ... , 2] )
    yB = K.minimum( target_boxes[ ... , 3], pred_boxes[ ... , 3] )
    interArea = K.maximum( 0.0 , xB - xA ) * K.maximum( 0.0 , yB - yA )
    boxAArea = (target_boxes[ ... , 2] - target_boxes[ ... , 0]) * (target_boxes[ ... , 3] - target_boxes[ ... , 1])
    boxBArea = (pred_boxes[ ... , 2] - pred_boxes[ ... , 0]) * (pred_boxes[ ... , 3] - pred_boxes[ ... , 1])
    iou = interArea / ( boxAArea + boxBArea - interArea )
    return iou

def iou_metric( y_true , y_pred ):
    return calculate_iou( y_true , y_pred )

def custom_loss( y_true , y_pred ):
    mse = tf.losses.mean_squared_error( y_true , y_pred ) 
    iou = calculate_iou( y_true , y_pred ) 
    return mse + ( 1 - iou )

In [81]:
num_classes = 3
alpha = 0.2
pred_vector_length = 4 + num_classes

model_layers = [
    keras.layers.Conv2D(16, kernel_size = (3,3), strides = 1, input_shape = input_shape),
    keras.layers.LeakyReLU(alpha),
    keras.layers.Conv2D(16, kernel_size = (3,3), strides = 1),
    keras.layers.LeakyReLU(alpha),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    keras.layers.Conv2D(32, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU(alpha=alpha),
    keras.layers.Conv2D(32, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.MaxPooling2D( pool_size=( 2 , 2 ) ),
    
    keras.layers.Conv2D(64, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Conv2D(64, kernel_size=(3, 3), strides=1),
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.MaxPooling2D( pool_size=( 2 , 2 ) ),
    
    keras.layers.Flatten(),
    
    keras.layers.Dense( 640 ) ,
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Drop
    keras.layers.Dense( 480 ) , 
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Dense( 120 ) , 
    keras.layers.LeakyReLU( alpha=alpha ) ,
    keras.layers.Dense( 62 ) , 
    keras.layers.LeakyReLU( alpha=alpha ) ,
    
    keras.layers.Dense( pred_vector_length ),
    keras.layers.LeakyReLU( alpha=alpha ) 
]

model = keras.models.Sequential(model_layers)
model.compile(
    optimizer = 'adam',
    loss = custom_loss,
    metrics=[iou_metric]
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [84]:
model.fit(x_train, y_train, batch_size=20, epochs=10, verbose=1)

Train on 111 samples
Epoch 1/10
111/111 [==============================] - 10s 94ms/sample - loss: 1.2422 - iou_metric: 0.0215
Epoch 2/10
111/111 [==============================] - 9s 80ms/sample - loss: 0.9461 - iou_metric: 0.2495
Epoch 3/10
111/111 [==============================] - 9s 81ms/sample - loss: 0.7631 - iou_metric: 0.3867
Epoch 4/10
111/111 [==============================] - 9s 79ms/sample - loss: 0.6964 - iou_metric: 0.4414
Epoch 5/10
111/111 [==============================] - 10s 91ms/sample - loss: 0.6481 - iou_metric: 0.4747
Epoch 6/10
111/111 [==============================] - 12s 107ms/sample - loss: 0.6053 - iou_metric: 0.5091
Epoch 7/10
111/111 [==============================] - 11s 97ms/sample - loss: 0.5323 - iou_metric: 0.5759
Epoch 8/10
111/111 [==============================] - 10s 87ms/sample - loss: 0.5242 - iou_metric: 0.5799
Epoch 9/10
111/111 [==============================] - 10s 88ms/sample - loss: 0.5340 - iou_metric: 0.5705
Epoch 10/10
111/111 [======

In [86]:
y_pred = model.predict(x_test)

In [87]:
y_pred[0]

array([0.11728775, 0.20961389, 0.8181522 , 0.82501984, 0.29319173,
       0.3376145 , 0.30370492], dtype=float32)

In [89]:
test_image = x_test[0] * 255.0

In [93]:
im = Image.fromarray(np.uint8(test_image*255))

In [95]:
im.show()